In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from datetime import timedelta
from pandas import Timedelta
import re

In [25]:
# Step 1: Load the data (replace 'your_file.csv' with your actual file)
df = pd.read_csv('/Users/MQ46079823/Downloads/welly test data/plate2_4%_EtOH.csv', header=0)

# Step 2: Time Parsing Function
def parse_time(time_obj):
    if isinstance(time_obj, float) or isinstance(time_obj, int):
        return float(time_obj) * 24 * 60  # Convert days to minutes
    elif isinstance(time_obj, pd.Timestamp):
        base_date = pd.Timestamp('1899-12-31')
        delta = time_obj - base_date
        return delta.total_seconds() / 60
    elif isinstance(time_obj, timedelta):
        return time_obj.total_seconds() / 60
    elif isinstance(time_obj, str):
        try:
            parts = time_obj.split(':')
            if len(parts) == 3:
                return timedelta(hours=int(parts[0]), minutes=int(parts[1]), seconds=int(parts[2])).total_seconds() / 60
            elif len(parts) == 2:
                return timedelta(hours=int(parts[0]), minutes=int(parts[1])).total_seconds() / 60
        except ValueError:
            return np.nan
    return np.nan

# Step 3: Apply time parsing to the 'Time' column
df['Time'] = df['Time'].apply(parse_time)
df.head()

,Time,sample1,sample1.1,sample1.2,A4,A5,A6,A7,A8,A9,...,H3,H4,H5,H6,H7,H8,H9,sample2,sample2.1,sample2.2
0,0.0,0.085,0.085,0.085,0.085,0.084,0.085,0.085,0.084,0.085,...,0.085,0.085,0.085,0.085,0.085,0.085,0.085,0.085,0.085,0.086
1,20.0,0.085,0.084,0.085,0.085,0.084,0.085,0.085,0.084,0.085,...,0.085,0.085,0.085,0.085,0.085,0.084,0.085,0.085,0.085,0.086
2,40.0,0.085,0.084,0.085,0.085,0.084,0.085,0.085,0.084,0.085,...,0.085,0.085,0.085,0.085,0.085,0.084,0.085,0.085,0.085,0.086
3,60.0,0.085,0.084,0.085,0.085,0.084,0.085,0.085,0.084,0.085,...,0.085,0.085,0.085,0.085,0.085,0.084,0.085,0.085,0.085,0.086
4,80.0,0.085,0.084,0.085,0.085,0.084,0.085,0.085,0.084,0.085,...,0.085,0.085,0.085,0.085,0.085,0.084,0.085,0.085,0.085,0.086


In [26]:
# Step 4: Assign Well Labels Function (to use in the next step)
def assign_well_labels(df, plate_type):
    if plate_type == '96':
        well_labels = [f"{chr(65 + i)}{j + 1}" for i in range(8) for j in range(12)]
    elif plate_type == '384':
        well_labels = [f"{chr(65 + i)}{j + 1}" for i in range(16) for j in range(24)]
    else:
        raise ValueError("Invalid plate type")
    
    df.columns = ['Time'] + well_labels[:len(df.columns) - 1]  # Adjust columns
    return df

# Example usage
df = assign_well_labels(df.copy(), '96')  # or '384' depending on your plate type
df

,Time,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,H3,H4,H5,H6,H7,H8,H9,H10,H11,H12
0,0.0,0.085,0.085,0.085,0.085,0.084,0.085,0.085,0.084,0.085,...,0.085,0.085,0.085,0.085,0.085,0.085,0.085,0.085,0.085,0.086
1,20.0,0.085,0.084,0.085,0.085,0.084,0.085,0.085,0.084,0.085,...,0.085,0.085,0.085,0.085,0.085,0.084,0.085,0.085,0.085,0.086
2,40.0,0.085,0.084,0.085,0.085,0.084,0.085,0.085,0.084,0.085,...,0.085,0.085,0.085,0.085,0.085,0.084,0.085,0.085,0.085,0.086
3,60.0,0.085,0.084,0.085,0.085,0.084,0.085,0.085,0.084,0.085,...,0.085,0.085,0.085,0.085,0.085,0.084,0.085,0.085,0.085,0.086
4,80.0,0.085,0.084,0.085,0.085,0.084,0.085,0.085,0.084,0.085,...,0.085,0.085,0.085,0.085,0.085,0.084,0.085,0.085,0.085,0.086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,4240.1,0.085,0.085,0.085,0.085,0.085,0.085,0.085,0.085,0.085,...,0.086,0.085,0.085,0.085,0.085,0.085,0.085,0.085,0.085,0.086
213,4260.1,0.085,0.085,0.085,0.085,0.085,0.085,0.085,0.085,0.085,...,0.086,0.085,0.085,0.085,0.085,0.085,0.085,0.085,0.085,0.086
214,4280.1,0.085,0.085,0.085,0.085,0.085,0.085,0.085,0.085,0.085,...,0.086,0.085,0.085,0.085,0.085,0.086,0.085,0.085,0.085,0.086
215,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
def calculate_max_growth_rate_per_hour(df):
    growth_rates = pd.DataFrame(columns=df.columns)
    
    for col in df.columns:
        if col == 'Time':
            continue
        
        # Calculate time and OD differences
        df['Time_diff'] = df['Time'].diff() / 60  # Time difference in hours (assuming 'Time' is in minutes)
        df[f'OD_diff_{col}'] = df[col].diff()  # OD difference
        
        # Calculate growth rate (OD change per hour)
        df[f'Growth_rate_{col}'] = df[f'OD_diff_{col}'] / df['Time_diff']
        
        # Filter out negative growth rates
        positive_growth_rates = df[df[f'Growth_rate_{col}'] > 0][f'Growth_rate_{col}']
        
        # Store the max growth rate for the current column
        if not positive_growth_rates.empty:
            max_growth_rate = positive_growth_rates.max()
            growth_rates = pd.concat([growth_rates, pd.DataFrame({'Sample': [col], 'Max_growth_rate': [max_growth_rate]})], ignore_index=True)
    
    return growth_rates


In [42]:
max_growth_rates

,Time,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,H5,H6,H7,H8,H9,H10,H11,H12,Sample,Max_growth_rate
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1,0.000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A2,0.003
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A3,0.000
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A4,0.000
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A5,0.003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H8,0.003
92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H9,0.000
93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H10,0.000
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H11,0.000


In [29]:
replicate_labels = {
    'A1': 'Sample_1',
    'A2': 'Sample_1',
    'A3': 'Sample_1',  # Treat A1 and A2 as replicates
    'B1': 'Sample_2',
    'B2': 'Sample_2'   # Treat B1 and B2 as replicates
}

In [32]:
# Step 8: Group replicates and calculate mean and std for growth rates
def group_replicates_and_calculate_mean_std(max_growth_rate_df, labels):
    grouped_growth_rates = {}
    
    for _, row in max_growth_rate_df.iterrows():
        col = row['Sample']
        growth_rate = row['Max_growth_rate']
        label = labels.get(col, col)  # Use the label to group replicates
        if label not in grouped_growth_rates:
            grouped_growth_rates[label] = []
        grouped_growth_rates[label].append(growth_rate)
    
    # Calculate mean and std for each group
    mean_std_summary = pd.DataFrame(columns=['Sample', 'Mean_growth_rate', 'Std_growth_rate'])
    for label, rates in grouped_growth_rates.items():
        mean_rate = np.mean(rates)
        std_rate = np.std(rates)
        mean_std_summary = pd.concat([mean_std_summary, pd.DataFrame({'Sample': [label], 'Mean_growth_rate': [mean_rate], 'Std_growth_rate': [std_rate]})], ignore_index=True)
    
    return mean_std_summary

# Step 9: Group by replicates and calculate mean and std
summary = group_replicates_and_calculate_mean_std(max_growth_rates, replicate_labels)

/var/folders/hp/j49c4d2x38d_33_48lldbqtm0000gr/T/ipykernel_93042/3565590230.py:18: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [43]:
# Step 10: Plotting function
def plot_max_growth_rate(summary):
    fig = go.Figure([go.Bar(
        x=summary['Sample'],
        y=summary['Mean_growth_rate'],
        error_y=dict(type='data', array=summary['Std_growth_rate'], visible=True),
        text=summary['Mean_growth_rate'],
        textposition='auto'
    )])
    
    fig.update_layout(
        title='Max Growth Rate (Mean ± Std)',
        xaxis_title='Sample',
        yaxis_title='Max Growth Rate (OD/hour)',
        paper_bgcolor='white',
        plot_bgcolor='white',
        width=800,
        height=600
    )
    
    return fig

# Step 11: Plot the results
fig = plot_max_growth_rate(summary)
fig.show()